In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

#### Load data & Have a first look

In [37]:
train_and_val = pd.read_excel('/Users/trongphan/Downloads/Rice_Datathon/data/training.xlsx')

In [38]:
train_and_val

,Date,Vehicle Category,GVWR Class,Fuel Type,Model Year,Fuel Technology,Electric Mile Range,Number of Vehicles Registered at the Same Address,Region,Vehicle Population
0,2019,P,Not Applicable,Gasoline,2020.0,ICE,Not Applicable,≥4,Statewide,395883
1,2020,P,Not Applicable,Gasoline,2020.0,ICE,Not Applicable,1,Statewide,370954
2,2021,P,Not Applicable,Gasoline,2020.0,ICE,Not Applicable,1,Statewide,349406
3,2019,P,Not Applicable,Gasoline,2019.0,ICE,Not Applicable,≥4,Statewide,348475
4,2019,P,Not Applicable,Gasoline,2018.0,ICE,Not Applicable,≥4,Statewide,333296
...,...,...,...,...,...,...,...,...,...,...
41048,2019,B,Not Applicable,Diesel,1983.0,ICE,Not Applicable,1,Statewide,1
41049,2019,B,Not Applicable,Diesel,1980.0,ICE,Not Applicable,2,Statewide,1
41050,2019,B,Not Applicable,Diesel,1978.0,ICE,Not Applicable,3,Statewide,1
41051,2019,B,Not Applicable,Diesel,1976.0,ICE,Not Applicable,2,Statewide,1


In [39]:
def summarize_df(df, max_columns=10):
    summary = {
        'shape': df.shape,
        'columns': df.columns.tolist()[:max_columns] + ['...'] if len(df.columns) > max_columns else df.columns.tolist(),
        'missing_values': {col: df.isnull().sum()[col] for col in df.columns[:max_columns]} if len(df.columns) > max_columns else df.isnull().sum().to_dict(),
        'describe': df.describe(include='all').iloc[:, :max_columns].to_dict() if len(df.columns) > max_columns else df.describe(include='all').to_dict(),
        'num_columns': len(df.columns),
        'distinct_values': {col: df[col].nunique() for col in df.columns[:max_columns]} if len(df.columns) > max_columns else {col: df[col].nunique() for col in df.columns},
        'columns_with_few_distinct_values': {col: df[col].unique().tolist() for col in df.columns[:max_columns] if df[col].nunique() < 20} if len(df.columns) > max_columns else {col: df[col].unique().tolist() for col in df.columns if df[col].nunique() < 20}
    }
    return summary


In [33]:
import json
summary = summarize_df(train_and_val)

with open('summary.json', 'w') as f:
    json.dump(summary, f, indent=4)


### Data Wrangling

In [40]:
# Convert 'Date' to datetime format and extract the year
train_and_val['Date'] = pd.to_datetime(train_and_val['Date'], format='%Y').dt.year

# Fill missing values in 'Model Year' with the median value
train_and_val['Model Year'].fillna(train_and_val['Model Year'].median(), inplace=True)

# Convert 'Model Year' to integer
train_and_val['Model Year'] = train_and_val['Model Year'].astype(int)

# Convert categorical columns to category dtype
categorical_columns = ['Vehicle Category', 'GVWR Class', 'Fuel Type', 'Fuel Technology', 'Electric Mile Range', 'Number of Vehicles Registered at the Same Address', 'Region']
for col in categorical_columns:
    train_and_val[col] = train_and_val[col].astype('category')

# Remove duplicates
train_and_val.drop_duplicates(inplace=True)

# Reset index
train_and_val.reset_index(drop=True, inplace=True)

# Display the cleaned dataframe
train_and_val = train_and_val.drop(columns=["Region"])

In [41]:
train_and_val

,Date,Vehicle Category,GVWR Class,Fuel Type,Model Year,Fuel Technology,Electric Mile Range,Number of Vehicles Registered at the Same Address,Vehicle Population
0,2019,P,Not Applicable,Gasoline,2020,ICE,Not Applicable,≥4,395883
1,2020,P,Not Applicable,Gasoline,2020,ICE,Not Applicable,1,370954
2,2021,P,Not Applicable,Gasoline,2020,ICE,Not Applicable,1,349406
3,2019,P,Not Applicable,Gasoline,2019,ICE,Not Applicable,≥4,348475
4,2019,P,Not Applicable,Gasoline,2018,ICE,Not Applicable,≥4,333296
...,...,...,...,...,...,...,...,...,...
41035,2019,B,Not Applicable,Diesel,1983,ICE,Not Applicable,1,1
41036,2019,B,Not Applicable,Diesel,1980,ICE,Not Applicable,2,1
41037,2019,B,Not Applicable,Diesel,1978,ICE,Not Applicable,3,1
41038,2019,B,Not Applicable,Diesel,1976,ICE,Not Applicable,2,1


In [42]:
train_and_val.to_csv('train_and_val.csv', index=False)

### List of model to test:
* Random Forest
* XGBoost 
* Decision Tree

## I. Dummy testing with Random Forest

In [103]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [115]:

# Load data (replace with your actual dataset)
# Assuming the data is in a CSV file named "data.csv"
# data = train_and_val[:1000]
data = train_and_val
# Preprocessing
# One-hot encode categorical columns
categorical_cols = ["Vehicle Category", "Fuel Type", "Fuel Technology", "Electric Mile Range"]
data = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

In [117]:
# Ordinal encode "Number of Vehicles Registered at the Same Address"
ordinal_mapping = {1: int(1), 2: int(2), 3: int(3), "\u22654": 4, "Unknown": -1}
data["Number of Vehicles Registered at the Same Address"] = data["Number of Vehicles Registered at the Same Address"].map(ordinal_mapping)

In [119]:
# Preprocessing
# Impute missing values in "Model Year"
imputer = SimpleImputer(strategy="median")
data["Model Year"] = imputer.fit_transform(data[["Model Year"]])
# Handle non-numeric values in "GVWR Class"
data["GVWR Class"] = data["GVWR Class"].replace({"Not Applicable": -1, "Unknown": -1})

In [120]:
# Feature engineering
data["Vehicle Age"] = data["Date"] - data["Model Year"]

In [121]:
# Check for missing values in the dataset
print("Missing values in each column:")
print(data.isnull().sum())

# Handle missing values in all columns
for col in data.columns:
    if data[col].dtype == "object" or pd.api.types.is_categorical_dtype(data[col]):  # Categorical columns
        data[col].fillna(data[col].mode()[0], inplace=True)
    else:  # Numerical columns
        data[col].fillna(data[col].median(), inplace=True)

Missing values in each column:
Date                                                 0
GVWR Class                                           0
Model Year                                           0
Number of Vehicles Registered at the Same Address    0
Vehicle Population                                   0
Vehicle Category_BS                                  0
Vehicle Category_BT                                  0
Vehicle Category_MC                                  0
Vehicle Category_MH                                  0
Vehicle Category_P                                   0
Vehicle Category_T1                                  0
Vehicle Category_T2                                  0
Vehicle Category_T3                                  0
Vehicle Category_T4                                  0
Vehicle Category_T5                                  0
Vehicle Category_T6                                  0
Vehicle Category_T7                                  0
Fuel Type_Electric                

In [122]:
# Split data
X = data.drop(columns=["Vehicle Population"])
y = data["Vehicle Population"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Train Random Forest model

model = RandomForestRegressor(random_state=42, n_estimators=100)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [126]:
# Evaluate
import math
y_pred = model.predict(X_test)
print("Mean Absolute Error (MAE):", mean_absolute_error(y_test, y_pred))
print("Mean Squared Error (MSE):", math.sqrt(mean_squared_error(y_test, y_pred)))
print("R-squared (R2):", r2_score(y_test, y_pred))

# Create a DataFrame with y_test and y_pred
results_df = pd.DataFrame({'y_test': y_test, 'y_pred': y_pred})
print(results_df)

Mean Absolute Error (MAE): 392.21646777099
Mean Squared Error (MSE): 2687.4109547313956
R-squared (R2): 0.9791228950851192
       y_test     y_pred
39468       1       2.17
34656       2      64.54
144    172682  152672.17
20510      39      57.81
12619     234     260.71
...       ...        ...
10406     377     350.27
29466       6      10.39
38677       1       1.36
32541       3       2.07
11153     321     326.05

[8208 rows x 2 columns]


### 1. Parameter tuning

#### a. Using GridSearchCV

In [130]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
import time

In [128]:
# Define the parameter grid
param_grid = {
    "n_estimators": [50, 100, 200],  # Number of trees
    "max_depth": [None, 10, 20, 30],  # Maximum depth of trees
    "min_samples_split": [2, 5, 10],  # Minimum samples to split a node
    "min_samples_leaf": [1, 2, 4],  # Minimum samples at a leaf node
    "max_features": ["sqrt", "log2"],  # Number of features to consider for splits
    "bootstrap": [True, False]  # Whether to use bootstrap sampling
}

# Initialize the Random Forest model
rf = RandomForestRegressor(random_state=42)

In [134]:
# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    scoring="neg_mean_squared_error",  # Use RMSE as the evaluation metric
    n_jobs=-1,  # Use all available CPU cores
    verbose=3  # Disable default verbose output
)

# Track time using tqdm
start_time = time.time()
with tqdm(total=len(grid_search.param_grid) * 5, desc="Grid Search Progress") as pbar:
    # Attach the callback to GridSearchCV
    grid_search.fit(X_train, y_train)

Grid Search Progress:   0%|          | 0/30 [00:00<?, ?it/s]

Fitting 5 folds for each of 432 candidates, totalling 2160 fits
[CV 1/5] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=-29463097.373 total time=   1.5s
[CV 2/5] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=-13835289.216 total time=   1.5s
[CV 4/5] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=-25912450.151 total time=   1.6s
[CV 3/5] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=-15013894.003 total time=   1.6s
[CV 5/5] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=-14408129.419 total time=   1.6s
[CV 1/5] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, sc

Grid Search Progress:   0%|          | 0/30 [00:27<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best R-squared Score:", grid_search.best_score_)

# Evaluate on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Test R-squared:", r2_score(y_test, y_pred))

#### b. Using Randomized GridSearch

In [136]:
from sklearn.model_selection import RandomizedSearchCV

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_grid,
    n_iter=50,  # Number of parameter combinations to try
    cv=5,
    scoring="r2",
    n_jobs=-1,
    verbose=3,
    random_state=42
)

# Fit the RandomizedSearchCV
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV 2/5] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=50;, score=0.833 total time=   0.5s
[CV 3/5] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=50;, score=0.825 total time=   0.5s
[CV 1/5] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=50;, score=0.789 total time=   0.5s
[CV 5/5] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=50;, score=0.871 total time=   0.5s
[CV 4/5] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=50;, score=0.768 total time=   0.6s
[CV 1/5] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=50;, score=0.857 total time=   1.0s
[CV 2/5] END bootstrap=Tru

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [None, 10, 20, 30],
                                        'max_features': ['sqrt', 'log2'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [50, 100, 200]},
                   random_state=42, scoring='r2', verbose=3)

In [137]:
# Print the best parameters and best score
print("Best Parameters:", random_search.best_params_)
print("Best R-squared Score:", random_searc`h.best_score_)

Best Parameters: {'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 30, 'bootstrap': False}
Best R-squared Score: 0.9565732866942425
